This notebook we will perform market basket analysis using association rule by apriori algorithm. To explore the items frequently purchased together. It allows retailers to identify relationships between the items that people buy. The result can be help retailers or convenience store make a decision for sales strategy further.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
%matplotlib inline
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_excel('Online Retail.xlsx')

In [3]:
## Drop null values and void transactions 
df.dropna(subset=['CustomerID'], inplace= True)
df = df[(df['Quantity']>0) & (df['UnitPrice']>0)]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


Let's generate the suitable table for calculation.

In [4]:
## Select features
df = df[['InvoiceNo', 'Description', 'Quantity', 'Country']]

In [5]:
## Grouping invoices and descriptions
grouping = df.groupby(['InvoiceNo', 'Description'])
basket = grouping['Quantity'].sum().reset_index()
## Convert quantity to 0 and 1 (0 = not buy & 1 = buy)
basket['Quantity'] = np.where(basket['Quantity'] >= 1, 1, basket['Quantity'])
basket = basket.pivot(index= 'InvoiceNo', columns= 'Description', values= 'Quantity')
## Replace null with 0
basket.fillna(0, inplace= True)
basket

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PC CUTLERY SET PANTRY DESIGN,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,...,YELLOW DRAGONFLY HELICOPTER,YELLOW EASTER EGG HUNT START POST,YELLOW FELT HANGING HEART W FLOWER,YELLOW FLOWERS FELT HANDBAG KIT,YELLOW GIANT GARDEN THERMOMETER,YELLOW METAL CHICKEN HEART,YELLOW PINK FLOWER DESIGN BIG BOWL,YELLOW POT PLANT CANDLE,YELLOW SHARK HELICOPTER,YELLOW/BLUE RETRO RADIO,YELLOW/ORANGE FLOWER DESIGN PLATE,YELLOW/PINK FLOWER DESIGN BIG MUG,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,YULETIDE IMAGES S/6 PAPER BOXES,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FINISH 15CM PLANTER POTS,ZINC FOLKART SLEIGH BELLS,ZINC HEART FLOWER T-LIGHT HOLDER,ZINC HEART LATTICE 2 WALL PLANTER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC PLANT POT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Calculate support values for each itemsets.

In [6]:
freq_itemsAP = apriori(basket, min_support= 0.02, use_colnames= True)
freq_itemsAP

,support,itemsets
0,0.021692,(3 STRIPEY MICE FELTCRAFT)
1,0.039175,(6 RIBBONS RUSTIC CHARM)
2,0.025146,(60 CAKE CASES VINTAGE CHRISTMAS)
3,0.035452,(60 TEATIME FAIRY CAKE CASES)
4,0.027034,(72 SWEETHEART FAIRY CAKE CASES)
...,...,...
238,0.023527,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY..."
239,0.024552,"(RED HANGING HEART T-LIGHT HOLDER, WHITE HANGI..."
240,0.022664,"(REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU..."
241,0.025254,"(WOODEN FRAME ANTIQUE WHITE , WOODEN PICTURE F..."


Mininig association rules and set lift to metric. Select only pairs of items where the lift is greater than 1. The result of association analysis shows which item is frequently purchased with other items.

In [7]:
rules_ap = association_rules(freq_itemsAP, metric= 'lift', min_threshold=1)
rules_ap =  rules_ap.sort_values(by= ['lift'], ascending= False)
rules_ap.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
74,(PINK REGENCY TEACUP AND SAUCER),"(GREEN REGENCY TEACUP AND SAUCER, ROSES REGENC...",0.030002,0.029193,0.021045,0.701439,24.027846,0.020169,3.251619
71,"(GREEN REGENCY TEACUP AND SAUCER, ROSES REGENC...",(PINK REGENCY TEACUP AND SAUCER),0.029193,0.030002,0.021045,0.720887,24.027846,0.020169,3.475290
73,(GREEN REGENCY TEACUP AND SAUCER),"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",0.037287,0.023527,0.021045,0.564399,23.989564,0.020167,2.241671
72,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.023527,0.037287,0.021045,0.894495,23.989564,0.020167,9.124846
9,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.030002,0.037287,0.024822,0.827338,22.188466,0.023703,5.575714


In [8]:
df[df['Description'].str.contains('ROSES REGENC')]

,InvoiceNo,Description,Quantity,Country
1077,536523,ROSES REGENCY TEACUP AND SAUCER,6,United Kingdom
1193,536530,ROSES REGENCY TEACUP AND SAUCER,4,United Kingdom
3397,536624,ROSES REGENCY TEACUP AND SAUCER,6,United Kingdom
3423,536627,ROSES REGENCY TEACUP AND SAUCER,12,United Kingdom
5292,536856,ROSES REGENCY TEACUP AND SAUCER,6,United Kingdom
...,...,...,...,...
534189,581135,ROSES REGENCY TEACUP AND SAUCER,24,United Kingdom
537799,581345,ROSES REGENCY TEACUP AND SAUCER,10,United Kingdom
539147,581433,ROSES REGENCY TEACUP AND SAUCER,6,EIRE
540213,581472,ROSES REGENCY TEACUP AND SAUCER,12,United Kingdom


As per the lift value from the associate rules table above. Show that pink regency teacup and saucer, green regency teacup and saucer and roses regency teacup and saucer are frequenly purchaesd together. Let's prove that it is true.

In [20]:
teacup = basket[(basket['ROSES REGENCY TEACUP AND SAUCER '] > 0) & 
                (basket['GREEN REGENCY TEACUP AND SAUCER'] > 0) & 
                (basket['PINK REGENCY TEACUP AND SAUCER'] > 0)]
print('Prove table No.1')
print(len(teacup[['ROSES REGENCY TEACUP AND SAUCER ', 
                 'GREEN REGENCY TEACUP AND SAUCER',
                 'PINK REGENCY TEACUP AND SAUCER']]))
teacup[['ROSES REGENCY TEACUP AND SAUCER ', 
                 'GREEN REGENCY TEACUP AND SAUCER',
                 'PINK REGENCY TEACUP AND SAUCER']]

Proof table 1
390


Description,ROSES REGENCY TEACUP AND SAUCER,GREEN REGENCY TEACUP AND SAUCER,PINK REGENCY TEACUP AND SAUCER
InvoiceNo,,,
538370,1.0,1.0,1.0
543099,1.0,1.0,1.0
543106,1.0,1.0,1.0
543116,1.0,1.0,1.0
543246,1.0,1.0,1.0
...,...,...,...
580727,1.0,1.0,1.0
580774,1.0,1.0,1.0
581433,1.0,1.0,1.0


In [19]:
teacup2 = basket[(basket['ROSES REGENCY TEACUP AND SAUCER '] > 0) & 
                (basket['GREEN REGENCY TEACUP AND SAUCER'] == 0) & 
                (basket['PINK REGENCY TEACUP AND SAUCER'] == 0)]
print('Prove table No.2')
print(len(teacup2[['ROSES REGENCY TEACUP AND SAUCER ', 
                 'GREEN REGENCY TEACUP AND SAUCER',
                 'PINK REGENCY TEACUP AND SAUCER']]))
teacup2[['ROSES REGENCY TEACUP AND SAUCER ', 
                 'GREEN REGENCY TEACUP AND SAUCER',
                 'PINK REGENCY TEACUP AND SAUCER']]

196


Description,ROSES REGENCY TEACUP AND SAUCER,GREEN REGENCY TEACUP AND SAUCER,PINK REGENCY TEACUP AND SAUCER
InvoiceNo,,,
536523,1.0,0.0,0.0
536530,1.0,0.0,0.0
536624,1.0,0.0,0.0
537254,1.0,0.0,0.0
537335,1.0,0.0,0.0
...,...,...,...
579093,1.0,0.0,0.0
579572,1.0,0.0,0.0
580173,1.0,0.0,0.0


From above table you will see that those three items has been purchased together up to 390 transactions. Next let's try to prove other items.

In [ ]:
alarm = basket[(basket['ALARM CLOCK BAKELIKE RED '] > 0) & (basket['ALARM CLOCK BAKELIKE GREEN'] > 0)]
print('Prove table No.3')
print(len(alarm[['ALARM CLOCK BAKELIKE RED ', 'ALARM CLOCK BAKELIKE GREEN']]))
alarm[['ALARM CLOCK BAKELIKE RED ', 'ALARM CLOCK BAKELIKE GREEN']]

530


Description,ALARM CLOCK BAKELIKE RED,ALARM CLOCK BAKELIKE GREEN
InvoiceNo,,
536370,1.0,1.0
536389,1.0,1.0
536395,1.0,1.0
536531,1.0,1.0
536599,1.0,1.0
...,...,...
581411,1.0,1.0
581436,1.0,1.0
581476,1.0,1.0


In [ ]:
lunch_box = basket[(basket['DOLLY GIRL LUNCH BOX'] > 0) & (basket['SPACEBOY LUNCH BOX '] > 0)]
print('Prove table No.4')
print(len(lunch_box[['DOLLY GIRL LUNCH BOX', 'SPACEBOY LUNCH BOX ']]))
lunch_box[['DOLLY GIRL LUNCH BOX', 'SPACEBOY LUNCH BOX ']]

424


Description,DOLLY GIRL LUNCH BOX,SPACEBOY LUNCH BOX
InvoiceNo,,
536412,1.0,1.0
536783,1.0,1.0
536784,1.0,1.0
537041,1.0,1.0
537111,1.0,1.0
...,...,...
580716,1.0,1.0
580873,1.0,1.0
581166,1.0,1.0


From above table you will see a pair of alarm clock bakelike red with alarm clock bakelike green and a pair of dolly girl lunch with box spaceboy lunch box. Their both found purchaesd together more than 400 times.

And next we will try to scope at germany. To explore germany customers behavior what item are frequently purchased together.

In [22]:
df_ger = df[df['Country'] == 'Germany']
grouping_ger = df_ger.groupby(['InvoiceNo', 'Description'])
basket_ger = grouping_ger['Quantity'].sum().reset_index()
basket_ger['Quantity'] = np.where(basket_ger['Quantity'] >= 1, 1, basket_ger['Quantity'])
basket_ger = basket_ger.pivot(index= 'InvoiceNo', columns= 'Description', values= 'Quantity')
basket_ger.fillna(0, inplace= True)
basket_ger

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 RED ROSE PEG PLACE SETTINGS,16 PIECE CUTLERY SET PANTRY DESIGN,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,200 RED + WHITE BENDY STRAWS,3 DRAWER ANTIQUE WHITE WOOD CABINET,3 HOOK HANGER MAGIC GARDEN,3 HOOK PHOTO SHELF ANTIQUE WHITE,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN GREEN AND CREAM,3 TIER CAKE TIN RED AND CREAM,3 TIER SWEETHEART GARDEN SHELF,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE SKULLS,...,WRAP BAD HAIR DAY,WRAP BILLBOARD FONTS DESIGN,WRAP BIRD GARDEN,WRAP CAROUSEL,WRAP CHRISTMAS SCREEN PRINT,WRAP CIRCUS PARADE,WRAP COWBOYS,WRAP DOILEY DESIGN,WRAP DOLLY GIRL,WRAP ENGLISH ROSE,WRAP FLOWER SHOP,WRAP FOLK ART,WRAP GINGHAM ROSE,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP MAGIC FOREST,WRAP MONSTER FUN,WRAP PAISLEY PARK,WRAP PINK FAIRY CAKES,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP RED DOILEY,WRAP RED VINTAGE DOILY,WRAP SUKI AND FRIENDS,WRAP VINTAGE LEAF DESIGN,WRAP VINTAGE PETALS DESIGN,WRAP WEDDING DAY,"WRAP, BILLBOARD FONTS DESIGN",YELLOW COAT RACK PARIS FASHION,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [23]:
freq_itemsAP_ger = apriori(basket_ger, min_support= 0.02, use_colnames= True)
freq_itemsAP_ger

,support,itemsets
0,0.024070,(10 COLOUR SPACEBOY PEN)
1,0.021882,(12 PENCIL SMALL TUBE WOODLAND)
2,0.032823,(3 HOOK HANGER MAGIC GARDEN)
3,0.041575,(3 PIECE SPACEBOY COOKIE CUTTER SET)
4,0.024070,(36 PENCILS TUBE RED RETROSPOT)
...,...,...
910,0.021882,"(SET OF 2 TEA TOWELS APPLE AND PEARS, ROUND SN..."
911,0.035011,"(ROUND SNACK BOXES SET OF 4 FRUITS , POSTAGE, ..."
912,0.024070,"(ROUND SNACK BOXES SET OF 4 FRUITS , POSTAGE, ..."
913,0.035011,"(ROUND SNACK BOXES SET OF 4 FRUITS , POSTAGE, ..."


In [24]:
rules_ger = association_rules(freq_itemsAP_ger, metric= 'lift', min_threshold=1)
rules_ger.sort_values(by= ['lift'], ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
208,(DOLLY GIRL CHILDRENS CUP),(DOLLY GIRL CHILDRENS BOWL),0.024070,0.026258,0.024070,1.000000,38.083333,0.023438,inf
209,(DOLLY GIRL CHILDRENS BOWL),(DOLLY GIRL CHILDRENS CUP),0.026258,0.024070,0.024070,0.916667,38.083333,0.023438,11.711160
1001,"(DOLLY GIRL CHILDRENS CUP, POSTAGE)",(DOLLY GIRL CHILDRENS BOWL),0.021882,0.026258,0.021882,1.000000,38.083333,0.021307,inf
1004,(DOLLY GIRL CHILDRENS BOWL),"(DOLLY GIRL CHILDRENS CUP, POSTAGE)",0.026258,0.021882,0.021882,0.833333,38.083333,0.021307,5.868709
1002,"(DOLLY GIRL CHILDRENS BOWL, POSTAGE)",(DOLLY GIRL CHILDRENS CUP),0.024070,0.024070,0.021882,0.909091,37.768595,0.021302,10.735230
1003,(DOLLY GIRL CHILDRENS CUP),"(DOLLY GIRL CHILDRENS BOWL, POSTAGE)",0.024070,0.024070,0.021882,0.909091,37.768595,0.021302,10.735230
103,(PINK VINTAGE SPOT BEAKER),(BLUE VINTAGE SPOT BEAKER),0.024070,0.030635,0.024070,1.000000,32.642857,0.023333,inf
102,(BLUE VINTAGE SPOT BEAKER),(PINK VINTAGE SPOT BEAKER),0.030635,0.024070,0.024070,0.785714,32.642857,0.023333,4.554340
2349,(RED SPOT CERAMIC DRAWER KNOB),"(POSTAGE, WHITE SPOT RED CERAMIC DRAWER KNOB, ...",0.030635,0.028446,0.021882,0.714286,25.109890,0.021010,3.400438
2340,"(POSTAGE, WHITE SPOT RED CERAMIC DRAWER KNOB, ...",(RED SPOT CERAMIC DRAWER KNOB),0.028446,0.030635,0.021882,0.769231,25.109890,0.021010,4.200584


In [25]:
dolly = basket_ger[(basket_ger['DOLLY GIRL CHILDRENS CUP'] == 1) & (basket_ger['DOLLY GIRL CHILDRENS BOWL'] == 1)]
print('Prove table No.5')
print(len(dolly[['DOLLY GIRL CHILDRENS CUP', 'DOLLY GIRL CHILDRENS BOWL']]))
dolly[['DOLLY GIRL CHILDRENS CUP', 'DOLLY GIRL CHILDRENS BOWL']]

Prove table No.5
11


Description,DOLLY GIRL CHILDRENS CUP,DOLLY GIRL CHILDRENS BOWL
InvoiceNo,,
558628,1.0,1.0
559362,1.0,1.0
560590,1.0,1.0
565416,1.0,1.0
565442,1.0,1.0
568103,1.0,1.0
569861,1.0,1.0
570640,1.0,1.0
574097,1.0,1.0


In [26]:
baker = basket_ger[(basket['PINK VINTAGE SPOT BEAKER'] == 1) & (basket_ger['BLUE VINTAGE SPOT BEAKER'] == 1)]
print('Prove table No.6')
print(len(baker[['PINK VINTAGE SPOT BEAKER', 'BLUE VINTAGE SPOT BEAKER']]))
baker[['PINK VINTAGE SPOT BEAKER', 'BLUE VINTAGE SPOT BEAKER']]

Prove table No.6
11


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Description,PINK VINTAGE SPOT BEAKER,BLUE VINTAGE SPOT BEAKER
InvoiceNo,,
558881,1.0,1.0
560569,1.0,1.0
560590,1.0,1.0
561037,1.0,1.0
561911,1.0,1.0
562370,1.0,1.0
564638,1.0,1.0
568103,1.0,1.0
569563,1.0,1.0


**Conclusion :** After we explore the basket of customers retail store can take the result from association rule to define sales strategy ,arrange the proper promotion or reduction in inventory levels. For instance, from prove table no.1 pink regency teacup and saucer, green regency teacup and saucer and roses regency teacup and saucer had been purchaesd together. If we look at prove table no.2 there are also have 196 customers that bought only roses regency teacup and saucer. The shop may be the shop may be arranged the promotion to increase revenue by recommend related product to customers. If some of products are frequently purchase together but did not appear close to each other the shop may be might be manage the new shelf layout.


**Credit :** 
*   https://pyshark.com/market-basket-analysis-using-association-rule-mining-in-python/


